In [61]:
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers
from spacy.tokens import DocBin
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
torch.cuda.is_available(), spacy.prefer_gpu()

(True, True)

In [48]:
!pip install spacy
!pip install spacy-transformers

In [49]:
!python -m spacy download en_core_web_trf

     -------------------------------------- 460.2/460.2 MB 6.3 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [64]:
# Reading the dataset
df = pd.read_csv("all-data.csv", encoding='latin-1', header = None)
df = df.rename(columns = {0:'Sentiment', 1:'Text'})

In [65]:
train = df.sample(frac = 0.8, random_state = 25)
test = df.drop(train.index)

In [62]:
nlp=spacy.load("en_core_web_trf")

In [66]:
train['tuples'] = train.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
train = train['tuples'].tolist()

In [67]:
test['tuples'] = test.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
test = test['tuples'].tolist()

In [68]:
def document(data):
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
    text.append(doc)
  
  return(text)

In [69]:
start_time = datetime.now()

#passing the train dataset into function 'document'
train_docs = document(train)

#Creating binary document using DocBin function in spaCy
doc_bin = DocBin(docs = train_docs)

#Saving the binary document as train.spacy
doc_bin.to_disk("train.spacy")
end_time = datetime.now()

#Printing the time duration for train dataset
print('Duration: {}'.format(end_time - start_time))

AttributeError: 'RobertaTokenizer' object has no attribute 'pipe'

In [57]:
# Calculate the time for converting into binary document for test dataset

start_time = datetime.now()

#passing the test dataset into function 'document'
test_docs = document(test)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")
end_time = datetime.now()

#Printing the time duration for test dataset
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:02.881832


In [58]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg


[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [59]:
start_time = datetime.now()

!python -m spacy train config.cfg --verbose  --gpu-id 0 --output ./output_updated

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

[i] Saving to output directory: output_updatedDuration: 0:10:53.937643

[i] Using GPU: 0

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['transformer', 'textcat']
[i] Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  -------------  ------------  ----------  ------
  0       0           0.00          0.03        0.00    0.00
  3     200           0.07         39.03       82.20    0.82
  6     400           0.26         10.96       86.50    0.86
 10     600           0.47          4.89       84.21    0.84
 13     800           0.09          0.88       85.51    0.86
 17    1000           0.03          0.42       84.70    0.85
 20    1200           0.04          0.19       84.33    0.84
 24    1400           0.03          0.29       84.70    0.85
 27    1600           0.06          0.04       82

[2022-04-10 02:00:31,269] [INFO] Set up nlp object from config
[2022-04-10 02:00:31,274] [DEBUG] Loading corpus from path: valid.spacy
[2022-04-10 02:00:31,275] [DEBUG] Loading corpus from path: train.spacy
[2022-04-10 02:00:31,275] [INFO] Pipeline: ['transformer', 'textcat']
[2022-04-10 02:00:31,277] [INFO] Created vocabulary
[2022-04-10 02:00:31,278] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical

In [60]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp("Adjusted for changes in the Group structure , the Division 's net sales increased by 1.7 % .")
print(demo.cats)

{'positive': 0.9436200857162476, 'negative': 0.0033362458925694227, 'neutral': 0.0530436746776104}
